In [ ]:
import numpy as np
import pytreenet as ptn

In [ ]:
d = 2
shapes = {
    (0, 0): (2, d),
    (0, 1): (2, 2, d),
    (0, 2): (2, 2, d),
    (0, 3): (2, 2, d),
    (0, 4): (2, 2, d),
    (0, 5): (2, 2, d),
    (0, 6): (2, 2, d),
    (0, 7): (2, 2, d),
    (0, 8): (2, 2, d),
    (0, 9): (2, d),}

sites = {
    (0, j): ptn.random_tensor_node(shapes[(0, j)], identifier=f"Site({0},{j})") for j in range(10)
}

ttn = ptn.TreeTensorNetworkState()

ttn.add_root(sites[(0, 0)][0], sites[(0, 0)][1])

connections = [
    ((0, 0), (0, 1), 0, 0),
    ((0, 1), (0, 2), 1, 0),
    ((0, 2), (0, 3), 1, 0),
    ((0, 3), (0, 4), 1, 0),
    ((0, 4), (0, 5), 1, 0),
    ((0, 5), (0, 6), 1, 0),
    ((0, 6), (0, 7), 1, 0),
    ((0, 7), (0, 8), 1, 0),
    ((0, 8), (0, 9), 1, 0)]

for (parent, child, parent_leg, child_leg) in connections:
    parent_id = f"Site({parent[0]},{parent[1]})"
    child_id = f"Site({child[0]},{child[1]})"
    ttn.add_child_to_parent(sites[child][0], sites[child][1], child_leg, parent_id, parent_leg)

In [ ]:
black_state = np.array([0,1])
white_state = np.array([1,0])
ttn = ptn.alternating_product_state(ttn, black_state, white_state, bond_dim = 1,  pattern = "checkerboard" , lattice_dim= 1)

In [ ]:
t = 2
u = 0.1
mu = 1
Lx = 1
Ly = 10

H1 = ptn.BoseHubbard_ham(t, u, mu, Lx, Ly, d)
H1 = H1.pad_with_identities(ttn , symbolic= True)
ttno = ptn.TTNO.from_hamiltonian(H1, ttn)

node_id_list = list(ttn.nodes.keys())
N_local_list = []
for node_id in node_id_list:
    N_local = ptn.Number_op_local(node_id, d)
    N_local = N_local.pad_with_identities(ttn, symbolic= True)
    N_local = ptn.TTNO.from_hamiltonian(N_local, ttn)
    N_local_list.append(N_local)
print(node_id_list)    

# Exact

In [ ]:
t = 2
u = 0.1
mu = 1
Lx = 1
Ly = 10
n_bosons = 2
system = ptn.BoseHubbard2D(Lx, Ly, n_bosons, t, u, mu)

# Create initial state
black_state = np.array([0,1])
white_state = np.array([1,0])
psi0 = system.alternating_product_state(black_state, white_state, pattern= "checkerboard")

# Evolve system
end_time = 4
dt = 0.01
result = system.evolve_system(psi0, end_time=end_time, dt=dt)


In [ ]:
eval_time = 2
occupations, occ_times = system.occupation_results(eval_time, result)


time_steps = occupations.shape[0]
time = np.arange(time_steps)
occupations_flat = occupations.reshape(time_steps, -1)